<a href="https://colab.research.google.com/github/ayz1070/CapstoneStockModelingProject/blob/main/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9(%EC%88%98%EC%A0%95v9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader
!pip install pykrx
!pip install pyOpenSSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.3 MB/s eta 0:00:00


In [2]:
import FinanceDataReader as fdr
import yfinance as yf
import openpyxl
from pykrx import stock
import time
import datetime
import ssl
import requests
from bs4 import BeautifulSoup
from time import sleep

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

# Colab => 데이터 CSV파일 가져오기
from google.colab import files
uploaded = files.upload()
kospi = pd.read_excel("kospi100.xlsx", engine='openpyxl')

Saving kospi100.xlsx to kospi100.xlsx


In [3]:
# 오늘의 날짜를 "today2"에 저장
today=datetime.datetime.today().strftime("%Y-%m-%d")
today2=datetime.datetime.today().strftime("%Y%m%d")

# :n --> kospi100,xlsx 파일의 종목중 위에서부터 n개 종목명 가져옴
companies = kospi['Name'][:2].tolist()

stocks = fdr.StockListing('KOSPI') # 코스피
stocks=stocks[['Code','Name']]

# 주가 정보
merged_df = pd.merge(kospi, stocks, on='Name', how='left')
merged_df[merged_df['Code'].isnull()]
merged_df=merged_df.dropna(axis=0)

# 환율 데이터
start_date = '2015-01-01'
data = yf.download(['USDKRW=X'],start=start_date, end=today)
data=data[['Close']]
data =data.rename(columns={'Close':'환율'})

# 금리 데이터
apikey='EPJ2CONXWQOE8KWK59GV'
def EcosDownload(Statcode, Freq, Begdate, Enddate, Subcode1, Subcode2, Subcode3):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/%s/xml/kr/1/100000/%s/%s/%s/%s/%s/%s/%s/'%(apikey, Statcode, Freq, Begdate, Enddate, Subcode1, Subcode2, Subcode3)
    raw = requests.get(url)
    xml = BeautifulSoup(raw.text,'xml')

    # Pandas 데이터프레임으로 전환합니다.
    raw_data = xml.find_all("row")
    date_list = []
    value_list = []

    for item in raw_data:
        value = item.find('DATA_VALUE').text.encode('utf-8')
        date_str = item.find('TIME').text
        value = float(value)
        date_list.append(datetime.datetime.strptime(date_str,'%Y%m%d'))
        value_list.append(value)

    df = pd.DataFrame(index = date_list)
    df['금리'] = value_list
    return df
df1 = EcosDownload('722Y001', 'D', '20150101', today2, '0101000','','')
df1.index = df1.index.rename('Date')

[*********************100%%**********************]  1 of 1 completed


In [4]:
# sequence_dataset 만들기
def make_sequence_dataset(feature, label, window_size):
  feature_list = []   # 생성될 feature list
  label_list = []     # 생성될 label list

  for i in range(len(feature)-window_size): # range는 전체값에서 window_size를 뺀 값
    feature_list.append(feature[i:i+window_size]) # feature list 에 i번째서 부터 window size 만큼의 입력데이터를 추가
    label_list.append(label[i+window_size]) # label list 에 그 다음 번째('window_size + 1' 번째)의 정답데이터를 추가
  return np.array(feature_list), np.array(label_list)

# 예측값 저장 dict 만들기
day_five = {}
day_ten = {}

In [5]:
# 종목별로 불러와서 예측하기
for company in companies:
  code=merged_df[merged_df['Name']== company]['Code'].values[0]
  stock_data = fdr.DataReader(code, start='2015-01-01', end=today)
  stock_data.reset_index(inplace=True)
  company_data=stock.get_market_fundamental_by_date(fromdate="20150101", todate=today2, ticker=code)
  company_data.reset_index(inplace=True)
  company_data.rename(columns = {'날짜':'Date'}, inplace=True)
  df=pd.merge(stock_data, company_data, on='Date', how='left')
  df=pd.merge(df, df1, on='Date', how='left')
  df=pd.merge(df, data, on='Date', how='left')
  df=df.fillna(method='ffill')

  csv_df = df.loc[:, ['Date', 'Close', 'BPS', 'EPS','금리', '환율']]
  csv_df = csv_df.set_index('Date') # 날짜를 index로 바꿈
  csv_df.dropna()

  # 날짜 빼고 정규화 작업 진행 -> MinMaxScaler 사용
  scaler = MinMaxScaler()

  # 정규화 수행 -> 정규화된 데이터프레임은 scaled_df
  scale_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
  scaled_df = scaler.fit_transform(csv_df[scale_cols]) # 정규화된 데이터는 넘파이 형태
  scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) # Pandas DataFrame 형태로 변경
  scaled_csv_df = scaled_df.set_index(csv_df.index)

  # 입력데이터 -> 이전날까지의 종가와 변수들
  # 정답데이터 -> 다음날의 종가
  feature_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
  label_cols = ['Close']

  # 입력데이터, 정답데이터 프레임 -> feature_df, label_df
  feature_df = scaled_csv_df[feature_cols]
  label_df = scaled_csv_df[label_cols]

  # DataFrame을 Numpy 형태로 저장
  feature_np = feature_df.to_numpy()
  label_np = label_df.to_numpy()

  window_size = 80  # window_size 만큼의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측
  X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

  # 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
  split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

  # 훈련data 는 전체 데이터의 70퍼센트
  X_train = X[0:split]
  y_train = y[0:split]

  # 테스트data 는 전체 데이터의 30퍼센트
  X_test = X[split:]
  y_test = y[split:]

  ## 5일 예측
  #LSTM 모델 구축
  model = Sequential()

  # 1차원 feature map 생성
  model.add(Conv1D(filters=32, kernel_size=5,
            padding="causal",
            activation="relu",
            input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

  # LSTM layer
  model.add(LSTM(units = 16, activation = 'tanh'))
  model.add(Dense(units = 16, activation = 'sigmoid'))
  model.add(Dense(units = 5)) # 출력층

  # 모델 컴파일
  loss = Huber()
  optimizer = Adam(0.0005)
  model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

  # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
  earlystopping = EarlyStopping(monitor='loss', patience=10)


  # 모델 학습 -> epoch은 100번 진행
  model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

  # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
  predictions = model.predict(X_test)

  # 실제값 변환
  # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
  close_min = csv_df['Close'].min()
  close_max = csv_df['Close'].max()

  # MinMaxScaler이용해서 실제값으로 역변환
  scaler2 = MinMaxScaler()
  scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

  # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
  scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
  scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

  # 예측한 출력값을 실제값으로 역변환
  original_pred_values = scaler2.inverse_transform(predictions)

  # 실제로 에측하기
  # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
  pred_feature = feature_df.tail(window_size)

  pred_feature_list = []
  pred_feature_list.append(pred_feature)
  pred_feature = np.array(pred_feature_list)

  # 5일 예측 -> 5일까지의 예측값 5개 출력
  predictions_5d = model.predict(pred_feature)
  pred_values_5d = scaler2.inverse_transform(predictions_5d)


  ##10일 예측
  #LSTM 모델 구축
  model = Sequential()

  # 1차원 feature map 생성
  model.add(Conv1D(filters=32, kernel_size=5,
            padding="causal",
            activation="relu",
            input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

  # LSTM layer
  model.add(LSTM(units = 16, activation = 'tanh'))
  model.add(Dense(units = 16, activation = 'sigmoid'))
  model.add(Dense(units = 10)) # 출력층

    # 모델 컴파일
  loss = Huber()
  optimizer = Adam(0.0005)
  model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

  # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
  earlystopping = EarlyStopping(monitor='loss', patience=10)


  # 모델 학습 -> epoch은 100번 진행
  model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

  # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
  predictions = model.predict(X_test)

  # 실제값 변환
  # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
  close_min = csv_df['Close'].min()
  close_max = csv_df['Close'].max()

  # MinMaxScaler이용해서 실제값으로 역변환
  scaler2 = MinMaxScaler()
  scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

  # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
  scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
  scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

  # 예측한 출력값을 실제값으로 역변환
  original_pred_values = scaler2.inverse_transform(predictions)

  # 실제로 에측하기
  # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
  pred_feature = feature_df.tail(window_size)

  pred_feature_list = []
  pred_feature_list.append(pred_feature)
  pred_feature = np.array(pred_feature_list)

  # 10일 예측 -> 10일까지의 예측값 10개 출력
  predictions_10d = model.predict(pred_feature)
  pred_values_10d = scaler2.inverse_transform(predictions_10d)

  #pred_values_5d = pred_values_5d.flatten().tolist()
  #pred_values_10d = pred_values_10d.flatten().tolist()

  close_5df = csv_df[['Close']] # 종가데이터
  close_5df.reset_index(inplace=True)
  close_5df.columns = ['Date', 'Close']  # 열 이름 변경
  close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환

  close_10df = csv_df[['Close']] # 종가데이터
  close_10df.reset_index(inplace=True)
  close_10df.columns = ['Date', 'Close']  # 열 이름 변경
  close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환

  pred5dCsvDf = pd.DataFrame(columns = ['Date', 'Close'])
  pred10dCsvDf = pd.DataFrame(columns = ['Date', 'Close'])

  # 5일 예측 날짜와 종가
  for i in range (len(pred_values_5d[0])):
    # 마지막 날짜를 찾고 그 다음 날짜 계산
    last_date = close_5df['Date'].max()
    next_date = last_date + pd.DateOffset(days=1)  # 하루를 더해 다음 날짜 계산
    # 새로운 데이터 생성 및 추가
    new_data = {'Date': [next_date], 'Close': [pred_values_5d[0,i]]}
    new_df = pd.DataFrame(new_data)
    pred5dCsvDf = pd.concat([pred5dCsvDf, new_df], ignore_index=True)
    # 그 다음 날짜 계산을 위한 마지막 날짜 변경
    close_5df = pd.concat([close_5df, new_df], ignore_index=True)

  # 10일 예측 날짜와 종가
  for i in range (len(pred_values_10d[0])):
    # 마지막 날짜를 찾고 그 다음 날짜 계산
    last_date = close_10df['Date'].max()
    next_date = last_date + pd.DateOffset(days=1)  # 하루를 더해 다음 날짜 계산
    # 새로운 데이터 생성 및 추가
    new_data = {'Date': [next_date], 'Close': [pred_values_10d[0,i]]}
    new_df = pd.DataFrame(new_data)
    pred10dCsvDf = pd.concat([pred10dCsvDf, new_df], ignore_index=True)
    # 그 다음 날짜 계산을 위한 마지막 날짜 변경
    close_10df = pd.concat([close_10df, new_df], ignore_index=True)

  # Date 열을 문자열로 다시 변환 (예: 'YYYY-MM-DD' 형식)
  pred5dCsvDf['Date'] = pred5dCsvDf['Date'].dt.strftime('%Y-%m-%d')
  pred10dCsvDf['Date'] = pred10dCsvDf['Date'].dt.strftime('%Y-%m-%d')

  day_five[company] = pred5dCsvDf.values.tolist()
  day_ten[company] = pred10dCsvDf.values.tolist()

  print(company,code)
  print('5일 예측값', pred_values_5d)
  print('10일 예측값', pred_values_10d)
  print('/n')

Epoch 1/100
46/46 [==============================] - 4s 28ms/step - loss: 0.1629 - mse: 0.3290
Epoch 2/100
46/46 [==============================] - 2s 35ms/step - loss: 0.0491 - mse: 0.0983
Epoch 3/100
46/46 [==============================] - 2s 45ms/step - loss: 0.0225 - mse: 0.0450
Epoch 4/100
46/46 [==============================] - 1s 27ms/step - loss: 0.0175 - mse: 0.0349
Epoch 5/100
46/46 [==============================] - 1s 27ms/step - loss: 0.0145 - mse: 0.0291
Epoch 6/100
46/46 [==============================] - 1s 27ms/step - loss: 0.0114 - mse: 0.0228
Epoch 7/100
46/46 [==============================] - 1s 27ms/step - loss: 0.0074 - mse: 0.0147
Epoch 8/100
46/46 [==============================] - 1s 28ms/step - loss: 0.0041 - mse: 0.0083
Epoch 9/100
46/46 [==============================] - 1s 27ms/step - loss: 0.0030 - mse: 0.0060
Epoch 10/100
46/46 [==============================] - 1s 28ms/step - loss: 0.0021 - mse: 0.0042
Epoch 11/100
46/46 [=============================

<ipython-input-5-2ada8c5aefa5>:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환
<ipython-input-5-2ada8c5aefa5>:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환


Epoch 1/100
8/8 [==============================] - 3s 47ms/step - loss: 0.1465 - mse: 0.2937
Epoch 2/100
8/8 [==============================] - 0s 45ms/step - loss: 0.1192 - mse: 0.2385
Epoch 3/100
8/8 [==============================] - 0s 43ms/step - loss: 0.0969 - mse: 0.1939
Epoch 4/100
8/8 [==============================] - 0s 48ms/step - loss: 0.0798 - mse: 0.1597
Epoch 5/100
8/8 [==============================] - 0s 36ms/step - loss: 0.0656 - mse: 0.1312
Epoch 6/100
8/8 [==============================] - 0s 29ms/step - loss: 0.0537 - mse: 0.1073
Epoch 7/100
8/8 [==============================] - 0s 29ms/step - loss: 0.0441 - mse: 0.0881
Epoch 8/100
8/8 [==============================] - 0s 28ms/step - loss: 0.0374 - mse: 0.0747
Epoch 9/100
8/8 [==============================] - 0s 28ms/step - loss: 0.0326 - mse: 0.0652
Epoch 10/100
8/8 [==============================] - 0s 27ms/step - loss: 0.0298 - mse: 0.0595
Epoch 11/100
8/8 [==============================] - 0s 28ms/step - lo

<ipython-input-5-2ada8c5aefa5>:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환
<ipython-input-5-2ada8c5aefa5>:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환




---



In [6]:
day_five

{'삼성전자': [['2023-09-22', 63507.37890625],
  ['2023-09-23', 62880.8515625],
  ['2023-09-24', 61536.74609375],
  ['2023-09-25', 62177.9375],
  ['2023-09-26', 63237.64453125]],
 'LG에너지솔루션': [['2023-09-22', 488711.4375],
  ['2023-09-23', 485581.875],
  ['2023-09-24', 494942.75],
  ['2023-09-25', 522234.25],
  ['2023-09-26', 490867.875]]}

In [7]:
day_ten

{'삼성전자': [['2023-09-22', 64283.6796875],
  ['2023-09-23', 62747.515625],
  ['2023-09-24', 65553.3203125],
  ['2023-09-25', 65396.3984375],
  ['2023-09-26', 67215.4765625],
  ['2023-09-27', 66289.40625],
  ['2023-09-28', 62844.28515625],
  ['2023-09-29', 67169.6953125],
  ['2023-09-30', 65918.0703125],
  ['2023-10-01', 64909.5390625]],
 'LG에너지솔루션': [['2023-09-22', 522466.34375],
  ['2023-09-23', 520864.0],
  ['2023-09-24', 526649.25],
  ['2023-09-25', 547730.5],
  ['2023-09-26', 530507.25],
  ['2023-09-27', 510826.0625],
  ['2023-09-28', 539706.9375],
  ['2023-09-29', 507908.28125],
  ['2023-09-30', 523129.375],
  ['2023-10-01', 527088.625]]}

In [ ]:
data = {
    'day_five': day_five
}

server_url = 'https://eb86-39-118-146-59.ngrok-free.app/day_five'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공


In [ ]:
data = {
    'day_ten': day_ten
}

server_url = 'https://eb86-39-118-146-59.ngrok-free.app/day_ten'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공
